In [52]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv('weather_and_count.csv')
df.head()

,Date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,...,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events,counts,DayOfWeek
0,2014-10-13,71,62.0,54,55,51,46,87,68,46,...,10,10,4,13,4,21,0.00,Rain,409,0
1,2014-10-14,63,59.0,55,52,51,50,88,78,63,...,10,9,3,10,5,17,0.11,Rain,491,1
2,2014-10-15,62,58.0,54,53,50,46,87,77,67,...,10,9,3,18,7,25,0.45,Rain,313,2
3,2014-10-16,71,61.0,52,49,46,42,83,61,36,...,10,10,10,9,4,-,0.00,Rain,395,3
4,2014-10-17,64,60.0,57,55,51,41,87,72,46,...,10,10,6,8,3,-,0.14,Rain,294,4


In [237]:
features = df.drop([
    'Events',
    'counts',
    'Date',
    'Max_Gust_Speed_MPH'
], axis=1).columns.values

target = 'counts'
validation_split = 0.2
dataset_size = df.shape[0]
features

array(['Max_Temperature_F', 'Mean_Temperature_F', 'Min_TemperatureF',
       'Max_Dew_Point_F', 'MeanDew_Point_F', 'Min_Dewpoint_F',
       'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea_Level_Pressure_In', 'Mean_Sea_Level_Pressure_In',
       'Min_Sea_Level_Pressure_In', 'Max_Visibility_Miles',
       'Mean_Visibility_Miles', 'Min_Visibility_Miles',
       'Max_Wind_Speed_MPH', 'Mean_Wind_Speed_MPH', 'Precipitation_In',
       'DayOfWeek'], dtype=object)

In [238]:
df_train = df.copy()
df_train = df_train[features]
df_train = df_train.fillna(df_train.mean())


In [239]:
split_at = dataset_size * (1-validation_split)
x_train = df_train.as_matrix() # or [0:split_at].as_matrix()
y_train = df[target].values # or [split_at:].as_matrix()

In [273]:
x_train

array([[ 71.  ,  62.  ,  54.  , ...,   4.  ,   0.  ,   0.  ],
       [ 63.  ,  59.  ,  55.  , ...,   5.  ,   0.11,   1.  ],
       [ 62.  ,  58.  ,  54.  , ...,   7.  ,   0.45,   2.  ],
       ..., 
       [ 81.  ,  68.  ,  55.  , ...,   4.  ,   0.  ,   0.  ],
       [ 70.  ,  64.  ,  57.  , ...,   9.  ,   0.  ,   1.  ],
       [ 71.  ,  65.  ,  59.  , ...,   8.  ,   0.  ,   2.  ]])

In [271]:
n_cols = len(features)  # or x_train.shape[1]
input_shape = (n_cols,)

# Specify the model
model = Sequential()
model.add(Dense(100, activation='relu', input_shape = input_shape))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='relu'))


# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience=5)

# Fit the model
model.fit(
    x_train,
    y_train,
    epochs=500,
    validation_split=validation_split
    #callbacks=[early_stopping_monitor]
)


Train on 551 samples, validate on 138 samples
Epoch 1/500
551/551 [==============================] - 2s - loss: 87028.7175 - acc: 0.0018 - val_loss: 30073.9701 - val_acc: 0.0000e+00
Epoch 2/500
551/551 [==============================] - 0s - loss: 30956.4023 - acc: 0.0054 - val_loss: 7178.0771 - val_acc: 0.0072
Epoch 3/500
551/551 [==============================] - 0s - loss: 25305.5730 - acc: 0.0036 - val_loss: 6547.0641 - val_acc: 0.0072
Epoch 4/500
551/551 [==============================] - 0s - loss: 21142.5104 - acc: 0.0018 - val_loss: 6486.3233 - val_acc: 0.0072
Epoch 5/500
551/551 [==============================] - 0s - loss: 17845.5683 - acc: 0.0018 - val_loss: 5642.7729 - val_acc: 0.0072
Epoch 6/500
551/551 [==============================] - 0s - loss: 13095.4444 - acc: 0.0036 - val_loss: 6323.0686 - val_acc: 0.0000e+00
Epoch 7/500
551/551 [==============================] - 0s - loss: 9446.9530 - acc: 0.0036 - val_loss: 11738.9804 - val_acc: 0.0000e+00
Epoch 8/500
551/551 [===

In [278]:
results = []
correct = wrong = 0

# iterate over all samples and use them as input for prediction (not recommended)
for i in range(0,dataset_size):
    pred = model.predict(np.array([df_train.iloc[i].as_matrix()]))
    is_ok = abs(pred[:,0][0] - df.iloc[i]['counts']) <= 70
    if is_ok:
        correct += 1
    else:
        wrong += 1
    #results.append((pred[:,0][0], df.iloc[i]['counts'], is_ok))

print('correct', correct, correct/dataset_size)
print('wrong', wrong, wrong/dataset_size)


correct 474 0.6879535558780842
wrong 215 0.31204644412191584


In [263]:
df_train.iloc[0].values
t = [ 90.  ,  82.  ,  72.  ,  60.  ,  55.  ,  51.  ,  99.  ,  78.  ,
        58.  ,  33.03,  32.79,  31.65,  11.  ,  11.  ,   5.  ,  15.  ,
         5.  ,   0.  ,   0.  ]

array([ 71.  ,  62.  ,  54.  ,  55.  ,  51.  ,  46.  ,  87.  ,  68.  ,
        46.  ,  30.03,  29.79,  29.65,  10.  ,  10.  ,   4.  ,  13.  ,
         4.  ,   0.  ,   0.  ])

In [274]:
pred = model.predict(np.array([[ 90.  ,  82.  ,  72.  ,  60.  ,  55.  ,  51.  ,  99.  ,  78.  ,
        58.  ,  33.03,  32.79,  31.65,  11.  ,  11.  ,   5.  ,  15.  ,
         5.  ,   0.  ,   6.  ]]))
pred[:,0][0]

520.74896

In [246]:
pred = model.predict(np.array([df_train.iloc[10].as_matrix()])) # 319
pred[:,0]

array([[ 282.08770752]], dtype=float32)

In [187]:
pred = model.predict(np.array([[60, 77, 0.33]])) # 323
pred

array([[ 228.07341003]], dtype=float32)

In [140]:
pred = model.predict(np.array([[59, 9, 0.11, 1]])) # 491
pred

array([[ 363.43496704]], dtype=float32)